In [3]:
# -*- coding:utf-8 -*-
'''
本例是对5MNIST_ShuZiShiBieWenTi的优化处理
part3: mnist_eval.py
主要用来测试神经网络
'''
import time
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import ComplexRebuildMNIST_part1 as mnist_inference
import ComplexRebuildMNIST_part2 as mnist_train

# 每10秒加载一次最新模型，并计算其正确率
EVAL_INTERVAL_SECS = 10

def evaluate(mnist):
    with tf.Graph().as_default() as g:
        # 定义输入输出的placeholder
        x = tf.placeholder(tf.float32, [None, mnist_inference.INPUT_NODE], name='x-input')
        y_ = tf.placeholder(tf.float32, [None, mnist_inference.OUTPUT_NODE], name='y-input')
        # 准备验证数据
        validate_feed = {x:mnist.validation.images, y_:mnist.validation.labels}
        
        # 计算前向传播，测试不关注正则化的值
        y = mnist_inference.inference(x, None, None)
        
        # 计算模型预测精度
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_,1))
        # 将比尔型转化为float32并求平均值，即得一组数据的正确率
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
        # 通过变量重命名的方式来加载模型
        # 这样在前向传播的过程中就不需要调用求滑动平均的函数来获取平均值
        # 这样就可以完全共用mnist_inference.py中定义的前向传播过程
        variable_averages = tf.train.ExponentialMovingAverage(mnist_train.MOVING_AVERAGE_DECAY)
        variables_to_restore = variable_averages.variables_to_restore()
        saver = tf.train.Saver(variables_to_restore)
        
        # 每隔EVAL_INTERVAL_SECS秒调用一次计算正确率的过程以检测训练过程中正确率的变化
        while True:
            with tf.Session() as sess:
                ckpt = tf.train.get_checkpoint_state(mnist_train.MODEL_SAVE_PATH)
                if ckpt and ckpt.model_checkpoint_path:
                    saver.restore(sess, ckpt.model_checkpoint_path)
                    global_step = ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
                    accuracy_score = sess.run(accuracy, feed_dict=validate_feed)
                    print('After %s training step(s), validation accuracy=%g'%(global_step, accuracy_score))
                else:
                    print('No checkpoint file found')
                    return
                time.sleep(EVAL_INTERVAL_SECS)
        
def main(argv=None):
    mnist = input_data.read_data_sets("MNIST_DATA", one_hot=True)
    evaluate(mnist)

if __name__ == '__main__':
    tf.app.run()

Extracting MNIST_DATA/train-images-idx3-ubyte.gz
Extracting MNIST_DATA/train-labels-idx1-ubyte.gz
Extracting MNIST_DATA/t10k-images-idx3-ubyte.gz
Extracting MNIST_DATA/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Restoring parameters from MNIST_MODEL/model.ckpt-5001
After 5001 training step(s), validation accuracy=0.9842
INFO:tensorflow:Restoring parameters from MNIST_MODEL/model.ckpt-5001
After 5001 training step(s), validation accuracy=0.9842
INFO:tensorflow:Restoring parameters from MNIST_MODEL/model.ckpt-5001
After 5001 training step(s), validation accuracy=0.9842
INFO:tensorflow:Restoring parameters from MNIST_MODEL/model.ckpt-5001
After 5001 training step(s), validation accuracy=0.9842
INFO:tensorflow:Restoring parameters from MNIST_MODEL/model.ckpt-5001
After 5001 training step(s), validation accuracy=0.9842
INFO:tensorflow:Restoring parameters from MNIST_MODEL/model.ckpt-5001
After 5001 training step(s), validation accuracy=0.9842
INFO:tensorflow:Restoring parameters from MNIST_MO

KeyboardInterrupt: 